In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools import tools
from src.model import OLSModel

In [ ]:
def calc_vif(dataf, features, include_dummies=False, include_constant=False, extra_feature_exclude=None, standardize=True):
    
    if extra_feature_exclude != None:
        features = [f for f in features if f not in extra_feature_exclude]
    
    dataf = dataf[features]
    
    if not include_dummies:
        features = [c for c in features if 'dummy' not in c]
        dataf = dataf[features]
    else:
        dataf = dataf[features]
    if include_constant:
        dataf = tools.add_constant(dataf)
        
    if standardize:
        scaler = StandardScaler()
        dataf = pd.DataFrame(scaler.fit_transform(dataf), columns=dataf.columns)
    
    df_vif = pd.DataFrame({
        'variable': dataf.columns,
        'vif': [variance_inflation_factor(dataf.values, i) for i in range(dataf.shape[1])]
    })
    
    return df_vif


def scientific_ols_table(model):
    
    dataf = pd.read_html(model.summary().tables[1].as_html(),header=0,index_col=0)[0]

    INDEX_SWAP = {
        'composite_relational_ig_practises': 'Composite Relational IG Practises',
        'composite_formal_ig_practises': 'Composite Formal IG Practises',
        'return_on_assets': 'Return on Assets',
        'log_n_employees': 'Number of employees (LOG)',
        'csr_index': 'CSR Index',
        'logmin_n_data_breaches': 'Number of Data Breaches (LOG, min = 1)',
        'sqrt_age_in_years': 'Age (Square Root)',
    }

    new_indices = []
    coefs = []
    stderr = []
    tvalue = []
    confintlow = []
    confinthigh = []
    pvalue = []
    for i, r in dataf.iterrows():

        if 'dummy' in i:
            new_indices.append(f"  {i.split('sector_')[1]}")
        else:
            try:
                new_indices.append(INDEX_SWAP[i])
            except:
                new_indices.append(i.title())

        if r['P>|t|'] <= 0.001:
            sig = '***'
        elif r['P>|t|'] <= 0.01:
            sig = '**'
        elif r['P>|t|'] <= 0.05:
            sig = '*'
        else:
            sig = ''

        coefs.append(('{:.3f}{}'.format(r['coef'], sig)))
        stderr.append(('{:.2f}'.format(r['std err'])))
        tvalue.append(('{:.2f}'.format(r['t'])))
        confintlow.append(('{:.2f}'.format(r['[0.025'])))
        confinthigh.append(('{:.2f}'.format(r['0.975]'])))
        confint = [f'{l} to {h}' for l, h in zip(confintlow, confinthigh)]
        pvalue.append(('{:.3f}'.format(r['P>|t|'])))

    df =  pd.DataFrame(np.column_stack([
        coefs,
        stderr,
        tvalue,
        confint,
        pvalue,
    ]), columns=[
        'Coefficient (b)',
        'SE',
        't statistic',
        '95% CI',
        'p value',
    ], index=new_indices)
    
    if model.f_pvalue <= 0.001:
        sig = '***'
    elif model.f_pvalue <= 0.01:
        sig = '**'
    elif model.f_pvalue <= 0.05:
        sig = '*'
    else:
        sig = ''

    dfextra = pd.DataFrame({
        'Coefficient (b)': [
            '{:.3f}'.format(model.rsquared),
            '{:.3f}'.format(model.rsquared_adj),
            '{:.3f}{}'.format(model.fvalue, sig),
            '{:.3f}'.format(model.f_pvalue),
        ]
    }, index=[
        'R2',
        'Adjusted R2',
        'F',
        'Probability F',
    ])
    
    return df, dfextra

## confs

In [ ]:
fixed_model_columns = [
    'firmhash',
    'firm',
    'sector',
    'young_firm',
    'reputation_score_2020',
]
Y_COLUMN = 'reputation_score_2020'

In [ ]:
df = pd.read_csv('../data/modelinput/information_governance_clean_dataset.csv')

## --> ols model by theory features

### - check on multicollinearity before running the model

In [ ]:
model_features = [c for c in df.columns if c not in fixed_model_columns]

In [ ]:
calc_vif(df,
         model_features,
#          extra_feature_exclude=['log_n_employees']
        )

In [ ]:
# model_features = [c for c in model_features if c not in ['log_n_employees']]
model = OLSModel(df, model_features, Y_COLUMN, scale=True)

In [ ]:
print(model.summary())

In [ ]:
df_ols, df_ols_extra = scientific_ols_table(model)

In [ ]:
df_ols

In [ ]:
df_ols_extra

In [ ]:
with pd.ExcelWriter('../data/modeloutput/scientific_ols_table.xlsx', mode='w') as writer:  
    df_ols.to_excel(writer, sheet_name='ols_betas')
    df_ols_extra.to_excel(writer, sheet_name='ols')

## --> PROCESS macro for moderation effect

In [ ]:
from pyprocessmacro import Process

In [ ]:
model = Process(data=df,
                model=1,
                x=['composite_relational_ig_practises'],
                y=Y_COLUMN,
                m=["young_firm"])
model.summary()

In [ ]:
model = Process(data=df,
                model=1,
                x=['composite_formal_ig_practises'],
                y=Y_COLUMN,
                m=["young_firm"])
model.summary()

## --> ols model by output of feature selection

In [ ]:
import json

In [ ]:
with open('../data/modeloutput/feature_fits_20210421_002632.json', 'r') as infile:
    results = json.load(infile)

In [ ]:
count = 0
max_score = 0
best_features = None
for k, v in results.items():
    if v['rsquared_adj'] > max_score:
        max_score = v['rsquared_adj']
        n_significant = v['n_significant']
        best_features = v['features']

In [ ]:
model = _model(df, best_features, Y_COLUMN)

In [ ]:
print(model.summary())